In [1]:
import myRNN_data as mrdata
import myRNN_model as mrmodel
import myRNN_pipline as mrpipl
import myS2S_pipline as mspipl
import torch,math,collections
import torch.nn as nn
from d2l import torch as d2l
import os,logging
# import matplotlib.pyplot as plt
LOG_FORMAT = "%(asctime)s-%(levelname)s-%(message)s"
logging.basicConfig(filename='./log/log.log', level=logging.INFO, format=LOG_FORMAT)


## 训练S2S模型-机器翻译数据

In [7]:
## 普通seq2seq
# model_path = './saved_models/nmt_eng2fra_s2s_pure'
# batch_size, num_steps = 64, 16
# embed_size, num_hiddens, num_layers, dropout = 128, 128, 2, 0.2
# incellplot = False
# epoch = 100
# num_examples = 100000 # 100000
# device = d2l.try_gpu()
# # model_path = train_pipline(model_path, batch_size, num_steps,
# #                            embed_size, num_hiddens, num_layers,
# #                            dropout,num_epochs=epoch, num_examples=num_examples, incellplot=incellplot)
# model_path = './saved_models/nmt_eng2fra_s2s_pure_bz64_ns16_dime128_dimh128_nl2_epo100_lr0.01_10w.pkl'
# mspipl.pridict_pipline(model_path, batch_size, num_steps, embed_size, num_hiddens, num_layers, dropout, device, num_examples=num_examples)

In [2]:
logging.debug("This is a debug log.")
logging.info("This is a info log.")
logging.warning("This is a warning log.")
logging.error("This is a error log.")
logging.critical("This is a critical log.")

In [3]:
train_iter, src_vocab, tgt_vocab, _, _ = mrdata.load_data_nmt(32, 10, num_examples=1000, is_split=False)

./data/fra-eng/fra_preprocessed.txt exists, loading...


## seq2seq attention 机制中实现attention 函数

### Attention 函数

In [ ]:
# 针对seq2seq的数据结构，实现注意力机制

# input arg: X, size = [batch_size, seq_len]
# input arg: en_outputs, size = [seq_len, batch_size, dim_hiddens]
# input arg: state, size = [num_layer *  num_direction, batch_size, dim_hiddens]
# input arg: x_vare_len, size = [batch_size]
X = self.emb(X).permute(1, 0, 2) # emb并调整维度顺序后 [seq_len, batch_size, dim_emb]

### attention
# Q = state; K = en_outputs; V = en_outputs
# res = F.softmax(Q.permute(1,0,2).matmul(K.permute(1,2,0))/math.sqrt(K.shape[-1])).matmul(K.permute(1,0,2))
# dim res = (batch_size, 1, dim_hiddens)
# 主要是维度变换的问题 对于批量矩阵相乘 只要保证前几维度相同即可自动计算后续例如size=(6,5,3,4) 和 size=(6,5,4,1)矩阵相乘得到(6,5,3,1)
atte_scores = state.permute(1,0,2).matmul(en_outputs.permute(1,2,0))
# 计算softmax分数
atte_scores = F.softmax(atte_scores/math.sqrt(en_outputs.shape[-1]), dim=-1)
atten_res = atte_scores.matmul(en_outputs.permute(1,0,2)).squeeze(1)
context = atten_res.repeat(X.shape[0], 1, 1)
# after cat size = [seq_len, batch_size, dim_hiddens + dim_emb]
X_and_context = torch.cat((X, context), 2)
output, state = self.rnn(X_and_context, state)
output = self.dense(output).permute(1, 0, 2)
# output的形状:(batch_size, num_steps, vocab_size)
# state的形状: (num_layers * num_direction, batch_size, num_hiddens)

### mask-softmax方法

In [38]:
def masked_softmax_my(score, valid_lens):
    # score: (batch_size, 1, seq_len)
    # valid_lens: (batch_size)
    if len(valid_lens)<=0:
        return softmax(score, dim=-1)
    batch_size, num_layers, seq_len = score.shape
    mat_valid = torch.Tensor(valid_lens).repeat(seq_len,1,1).permute(2,1,0)
    mat_arrage = torch.arange(seq_len).repeat(batch_size,1,1)
    score[mat_arrage>=mat_valid]=-1e6
    score = nn.functional.softmax(score, dim=-1)
    return score
    

### 训练seq2seq attention模型-翻译数据

In [ ]:
# model_path = './saved_models/nmt_eng2fra_s2s_atten'
# batch_size, num_steps = 64, 16
# embed_size, num_hiddens, num_layers, dropout = 128, 128, 2, 0.2
# incellplot = False
# epoch = 100
# num_examples = 10000
# device = d2l.try_gpu()
# model_path = train_s2s_attention_pipline(model_path, batch_size, num_steps,
#                            embed_size, num_hiddens, num_layers,
#                            dropout,num_epochs=epoch, num_examples=num_examples, incellplot=incellplot)
# model_path = './saved_models/nmt_eng2fra_s2s_atten_bz64_ns16_dime128_dimh128_nl2_epo200_lr0.01_1w.pkl'
# pridict_s2s_attention_pipline(model_path, batch_size, num_steps, embed_size, num_hiddens, num_layers, dropout, device, num_examples=num_examples)
# # 0.110 atten 32 32 10000
# # avg blue:0.281, test num:5000 nmt_eng2fra_s2s_atten_bz64_ns16_dime128_dimh128_nl1_epo200_lr0.01_10w.pkl